Based on the code shown in https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_sift.html

In [ ]:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.feature import SIFT, match_descriptors, plot_matched_features
from skimage import transform
from PIL import Image
import numpy as np
import os
import cv2
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform
from pathlib import Path

In [ ]:
# Load the datafiles and make them grayscale
def load_image(path):
    return rgb2gray(np.array(Image.open(path)))

In [ ]:
# Used to acces the name of the folder we are working on : not necessary
print("Répertoire courant :", os.getcwd())


In [ ]:
# Extract SIFT keypoints and descriptors
def extract_sift_features(image):
    sift = SIFT()
    sift.detect_and_extract(image)
    return sift.keypoints, sift.descriptors

Problem of dimensions for the first two pictures.

In [ ]:
def resize_to_match(img1, img2):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    return cv2.resize(img2, (w1, h1)) if (h1 != h2 or w1 != w2) else img2

In [ ]:
# Visualize the matches
def match_and_plot(img1, img2, keypoints1, descriptors1, keypoints2, descriptors2, title):
    matches = match_descriptors(descriptors1, descriptors2, max_ratio=0.6, cross_check=True)

    #img2 = resize_to_match(img1, img2)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    plt.gray()
    plot_matched_features(image0=img1, image1=img2, 
                          keypoints0=keypoints1, keypoints1=keypoints2,
                          matches=matches, ax=ax, 
                          keypoints_color='cyan',
                          only_matches=True)
    ax.axis('off')
    ax.set_title(title)
    plt.show()
    return matches

In [ ]:
# Compute matching accuracy using optional homography
def compute_matching_accuracy(keypoints1, keypoints2, matches, homography=None, threshold=5):
    """
    Compute the matching accuracy based on a homography.
    If no homography is provided, only the number of matches is printed.
    """
    if homography is None:
        print(f"{len(matches)} matches found (no validation)")
        return None

    if matches is None or len(matches) == 0:
        print("No matches to evaluate.")
        return 0

    correct = 0
    for i, j in matches:
        pt1 = np.array([*keypoints1[i], 1.0])  # homogeneous coordinates
        projected = homography @ pt1
        projected /= projected[2]  # normalize

        pt2 = keypoints2[j]
        error = np.linalg.norm(projected[:2] - pt2)

        if error < threshold:
            correct += 1

    accuracy = correct / len(matches) if matches.size > 0 else 0
    print(f"Matching Accuracy: {accuracy:.2%} ({correct}/{len(matches)} correct matches)")
    return accuracy

In [ ]:
def create_homography(keypoints1, keypoints2, descriptors1, descriptors2):
    """
    Create a homography matrix from source points to destination points.
    """
    matches = match_descriptors(descriptors1, descriptors2, max_ratio=0.6, cross_check=True)
    if matches.shape[0] < 4:
        print(f"Not enough matches to estimate homography: {matches.shape[0]} found")
        return None
    
    if matches is None:
        print("No matches found.")
        return None
    
    src = keypoints1[matches[:, 0]]  # points from image 1
    dst = keypoints2[matches[:, 1]]  # corresponding points in image 2

    model_robust, inliers = ransac(
        (src, dst),
        ProjectiveTransform,
        min_samples=4,
        residual_threshold=2,
        max_trials=1000
    )
    H = model_robust.params 
    return H, matches


In [ ]:


def batch_compare_tests_to_dataset(test_dir, dataset_dir):
    test_paths = sorted(Path(test_dir).glob("*"))
    dataset_paths = sorted(Path(dataset_dir).rglob("*"))
    
    for test_path in test_paths:
        print(f"\n🧪 Test image: {test_path.name}")
        img_test = load_image(str(test_path))
        keypoints_test, descriptors_test = extract_sift_features(img_test)

        for ref_path in dataset_paths:
            if ref_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif"]:
                continue

            img_ref = load_image(str(ref_path))
            keypoints_ref, descriptors_ref = extract_sift_features(img_ref)

            if descriptors_ref is None or descriptors_test is None:
                print(f"⚠️ Skipped {ref_path.name} due to missing descriptors")
                continue

            H, matches = create_homography(keypoints_ref, keypoints_test, descriptors_ref, descriptors_test)
            accuracy = compute_matching_accuracy(keypoints_ref, keypoints_test, matches, H)

            print(f"🔗 Accuracy of {ref_path.name} vs {test_path.name}: {accuracy:.2f}")


In [ ]:
batch_compare_tests_to_dataset("data/tests", "data/images")